In [1]:
import numpy as np
import timeit
import solver_lib
import matplotlib.pyplot as plt
import time
from scipy.integrate import solve_ivp
from scipy.linalg import solve
from scipy.interpolate import interp1d

In [10]:
# constants and parameters
n_c = 1000      # no. of Chebyshev nodes
nmax = 2000     # max vector length
nr_sp = 99999   # no. of spatial nodes for pressure solution
ntser = 1000    # no. of time steps in pressure model
ntrial = 20     # max number of trials for Newton-Raphson (at 21st, it gives up)
tol_NR = 1e-4   # tolerance for NR
n = n_c + 1     # to deal with index mismatch error with matvec

In [11]:
# Allocate necessary arrays
#Jacx = np.zeros((n_c + 1, n_c + 1))
Jacx = np.zeros((n, n))
rd_block = np.zeros((nr_sp, 3))  # adjusting dimensions
slip_out = np.zeros((n_c + 1, 5))
phi = np.zeros(n_c)
model = np.zeros(n_c + 1)
#ones_v = np.ones(n_c + 1)
ones_v = np.ones(n)

In [5]:
ltry = 1000.0       # NR param 1 (make this such that a number of values are checked through (in gs version of code))
phi1 = 1.0          # NR param 2

In [ ]:
# Define constants
alpha = 0.01
r0 = 0.0
P0 = 225.0
sigma1 = 6e6
sigma3 = 3e6
dip = 70.0
sigma = 300.0
Str_drop = 150.0
p_frac = P0/sigma
s_frac = Str_drop/sigma
fratio = 0.9
gamma = 0.02

In [12]:
def interp1D(x, y, xi):
    """Interpolate the 1D function."""
    f = interp1d(x, y, fill_value="extrapolate")
    return f(xi)

def matvec_MKL(matrix, vector):
    """Matrix-vector multiplication."""
    return np.dot(matrix, vector)

def read_unformatted(file_path, rec, dtype, shape):
    """
    Reads a specific record from a binary file.

    Args:
        file_path (str): Path to the binary file.
        rec (int): Record index to read (1-based index).
        dtype (data-type): The data type of elements in the record.
        shape (tuple): The shape of the data to read for one record.

    Returns:
        numpy.ndarray: An array containing the data from the specified record,
                       reshaped to the given shape. Returns None if reading fails.
    """
    record_size = np.product(shape) * np.dtype(dtype).itemsize  # size of one record in bytes
    offset = (rec - 1) * record_size                            # compute the offset for the record

    try:
        with open(file_path, 'rb') as file:
            file.seek(offset)                                               # move file pointer to start of reqd record
            data = np.fromfile(file, dtype=dtype, count=np.product(shape))  # read data
            if data.size != np.product(shape):                              # ensuring whole data is read
                raise ValueError("Not enough data in file to read the specified record.")
            data = data.reshape(shape)                                      # reshape the data
        return data
    except Exception as e:
        print(f"Failed to read the file: {e}")
        return None

In [13]:
# i/o file setup
input_pressure_file = 'pressure_profile.dat'
slip_output_file = 'dat_' + str(100*s_frac) + '_' + str(100*p_frac) + '_' + str(100*fratio) + '_' + str(100*gamma) + '.out'
dumpfile1 = 'dumpfile1.dat'
dumpfile2 = 'dumpfile2.dat'

In [ ]:
# main crack sim loop
for t_ct in range(4, ntser + 1):
    # reading pressure file lines
    rd_block = read_unformatted(input_pressure_file, rec=1+(t_ct-1), dtype=np.float64, shape=(nr_sp, 3))
    
    time_current = rd_block[0, 2]

    # update rad_v and pressure
    lnrad_v = rd_block[:, 0]
    h = np.abs(lnrad_v[1] - lnrad_v[0])
    rad_v = lnrad_v
    pres_sim = rd_block[:, 1]
    pres_grad = np.gradient(pres_sim, h)
    
    # conditions and loop control
    if t_ct > 2:
        L_e = r0 + np.sqrt(alpha * time_current)
    
    # NR code for model update
    for trial in range(1, ntrial + 1):
        r_crack = model[n_c] * np.linspace(-1, 1, n_c + 1)  # Scaled crack coordinates
        
        # interpolating pressure
        pres_interp = interp1D(rad_v, pres_sim, r_crack)
        pres_grad_interp = interp1D(rad_v, pres_grad, r_crack)
        
        # jacobian and function evaluation updates
        f_eval = -(Str_drop * ones_v - pres_interp / sigma + matvec_MKL(Jacx, model[:n_c+1]))
        
        # solving the linear system
        start = time.time()
        solution = solve(Jacx, f_eval)
        finish = time.time()
        
        # updating model
        model += solution
        time_total = finish - start

        # convergence check
        if np.max(np.abs(solution)) < tol_NR:
            print(f"Converged at time step {t_ct} after {trial} trials.")
            break
    
    #np.savetxt(slip_output_file, slip_out)
    np.save(slip_output_file, slip_out)

print("End of simulation.")